#### Импорт библиотек
random - библиотека для генерации случайных величин

networkx - библиотека для генерации/визуализации графов

matplotlib - библиотека для визуализации(графиков)

numpy - библиотека для математических операций

ipywidgets - библиотека для интерфейса

IPython.display - библиотека для управления выводом

In [1]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

#### Параметры для работы

In [2]:
state = {
    "graph": None,
    "edges": None,
    "population": None,
    "best_scores": [],
    "mean_scores": [],
    "generation": 0,
    "params": {
        "num_vertices": 10,
        "num_edges": 15,
        "pop_size": 20,
        "generations": 30,
        "mutation_rate": 0.1,
        "selection_method": "tournament"
    }
}

#### Функция для генерации неориентированного графа

In [3]:
def generate_graph(num_vertices, num_edges):
    G = nx.Graph()
    G.add_nodes_from(range(num_vertices))
    while G.number_of_edges() < num_edges:
        u, v = random.sample(range(num_vertices), 2)
        G.add_edge(u, v)
    return G

#### Функция для генерации хромосомы(вектор из 0/1)

In [4]:
def generate_individual(num_vertices):
    return [random.choice([0, 1]) for _ in range(num_vertices)]

#### Функция приспособленности

In [5]:
def fitness(individual, edges):
    uncovered = sum(1 for u, v in edges if not (individual[u] or individual[v]))
    penalty = 10
    return - (sum(individual) + penalty * uncovered)

#### Функция для генерации популяции

In [6]:
def generate_population(size, num_vertices):
    return [generate_individual(num_vertices) for _ in range(size)]

#### Функции для скрещивания и мутации

In [7]:
def crossover(p1, p2):
    point = random.randint(1, len(p1) - 1)
    return p1[:point] + p2[point:]

def mutate(ind, rate):
    return [(1 - g) if random.random() < rate else g for g in ind]


#### Функции турнирного и рулетного отбора

In [8]:
def tournament_selection(pop, scores):
    i1, i2 = random.sample(range(len(pop)), 2)
    return pop[i1] if scores[i1] > scores[i2] else pop[i2]

In [9]:
def roulette_selection(population, scores):
    min_score = min(scores)
    shifted = [s - min_score + 1e-6 for s in scores]
    total = sum(shifted)
    probs = [s / total for s in shifted]
    idx = np.random.choice(len(population), p=probs)
    return population[idx]

#### Функция для одного поколения

In [10]:
def step_ga():
    pop = state["population"]
    edges = state["edges"]
    m_rate = state["params"]["mutation_rate"]
    method = state["params"].get("selection_method", "tournament")
    scores = [fitness(ind, edges) for ind in pop]
    new_pop = []
    for _ in range(len(pop)):
        if method == 'roulette':
            p1 = roulette_selection(pop, scores)
            p2 = roulette_selection(pop, scores)
        else:
            p1 = tournament_selection(pop, scores)
            p2 = tournament_selection(pop, scores)
        child = crossover(p1, p2)
        child = mutate(child, m_rate)
        new_pop.append(child)
    state["population"] = new_pop
    state["best_scores"].append(max(scores))
    state["mean_scores"].append(np.mean(scores))
    state["generation"] += 1


#### Визуализация

In [11]:
def visualize_graph(G, cover):
    pos = nx.spring_layout(G, seed=42)
    color_map = ['lightgreen' if cover[node] else 'lightgrey' for node in G.nodes]
    plt.figure(figsize=(6, 5))
    nx.draw_networkx(G, pos, node_color=color_map, node_size=600, with_labels=True)
    plt.title("Вершинное покрытие")
    plt.axis("off")
    plt.show()

def plot_fitness():
    best = state["best_scores"]
    mean = state["mean_scores"]

    plt.figure(figsize=(7, 4))
    plt.plot(best, marker='o', label='Best fitness', color='royalblue', linewidth=2)
    plt.plot(mean, marker='s', label='Average fitness', color='darkorange', linewidth=2)
    plt.xlabel('Поколение')
    plt.ylabel('Fitness (больше — лучше)')
    plt.title("Динамика приспособленности")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()


#### Main core программы

In [12]:
def show_min_covers():
    pop = state["population"]
    min_size = min(sum(ind) for ind in pop)
    min_covers = [ind for ind in pop if sum(ind) == min_size]
    print(f"\n Минимальный размер покрытия: {min_size}")
    print("Список всех покрытий этого размера:")
    for cov in min_covers:
        print(cov)

vertex_input       = widgets.IntText(value=10, description="Вершины:")
edge_input         = widgets.IntText(value=15, description="Рёбра:")
pop_input          = widgets.IntText(value=20, description="Популяция:")
gen_input          = widgets.IntText(value=30, description="Поколения:")
mut_input          = widgets.FloatText(value=0.1, description="Мутация:")
skip_input         = widgets.IntText(value=5,  description="Шагов пропустить:")
selection_dropdown = widgets.Dropdown(
    options=[('Турнир', 'tournament'), ('Рулетка', 'roulette')],
    value='tournament',
    description='Селекция:'
)
edges_display      = widgets.Textarea(
    value='',
    description='Список рёбер:',
    layout=widgets.Layout(width='100%', height='100px'),
    disabled=True
)
start_btn          = widgets.Button(description="Сгенерировать граф")
step_btn           = widgets.Button(description="Следующий шаг")
skip_btn           = widgets.Button(description="Пропустить N поколений")
output_area        = widgets.Output()

def initialize(_):
    with output_area:
        clear_output()
        state["params"].update({
            "num_vertices":    vertex_input.value,
            "num_edges":       edge_input.value,
            "pop_size":        pop_input.value,
            "generations":     gen_input.value,
            "mutation_rate":   mut_input.value,
            "selection_method":selection_dropdown.value
        })
        G = generate_graph(vertex_input.value, edge_input.value)
        pop = generate_population(pop_input.value, vertex_input.value)
        state.update({
            "graph":       G,
            "edges":       list(G.edges()),
            "population":  pop,
            "best_scores": [],
            "mean_scores": [],
            "generation":  0
        })
        edges_display.value = '\n'.join(f"{u} — {v}" for u, v in state["edges"])
        print("Граф сгенерирован. Список рёбер обновлён.")

def _do_steps(count):
    for _ in range(count):
        step_ga()
    best = max(state["population"], key=lambda ind: fitness(ind, state["edges"]))
    covered = sum(1 for u, v in state["edges"] if best[u] or best[v])
    total   = len(state["edges"])
    size    = sum(best)
    fit_val = fitness(best, state["edges"])
    print(f"Поколение {state['generation']}")
    print(f"Лучшее решение: {best}")
    print(f"Размер покрытия: {size}")
    print(f"Покрыто рёбер: {covered}/{total} ({covered/total:.2%})")
    print(f"Fitness: {fit_val:.4f}")
    visualize_graph(state["graph"], best)
    plot_fitness()

def next_step(_):
    with output_area:
        clear_output()
        if state["generation"] < state["params"]["generations"]:
            _do_steps(1)
        else:
            print("Максимум поколений достигнут.")
            show_min_covers()

def skip_steps(_):
    with output_area:
        clear_output()
        to_run = min(skip_input.value, state["params"]["generations"] - state["generation"])
        if to_run <= 0:
            print("Поколения закончились.")
            show_min_covers()
        else:
            _do_steps(to_run)
            if state["generation"] == state["params"]["generations"]:
                show_min_covers()

start_btn.on_click(initialize)
step_btn.on_click(next_step)
skip_btn.on_click(skip_steps)

display(widgets.VBox([
    vertex_input, edge_input, pop_input, gen_input, mut_input,
    selection_dropdown,
    widgets.HBox([skip_input, skip_btn]),
    widgets.HBox([start_btn, step_btn]),
    edges_display,
    output_area
]))